In [172]:
import igl
import numpy as np
import torch
import open3d
from matplotlib import pyplot as plt
from numpy.random import default_rng

In [173]:
def readLbl(fileName):
    #takes in file name, returns the labels as an array
    file1 = open(fileName, 'r')
    Lines = file1.readlines()

    count = 0

    lbls = np.empty([600])
    # Strips the newline character
    for line in Lines:
        count += 1
        text = line.strip()[1:].split('.')
        text[1] = text[1].split(' ')[1]
        lbls[int(text[0])] = int(text[1])
        #print("Line{}: {}".format(count, )))
    return lbls

def lbl_2_lblMat(lbls):
    #takes in the labels as an array, returns an adjacency matrix between objects of same label
    labelMat = igl.all_pairs_distances(lbls,lbls,False)
    labelMat = np.where(labelMat > 0.5, 1, 0)
    return labelMat

def compareGroups(grpa,grpb,lbl,disSMat):
    #takes in two group indices, labels and dissimiliarity matrix, returns how similar they are as two groups
    grpA_idx = np.reshape(np.where(lbl == grpa),(20,))
    grpB_idx = np.reshape(np.where(lbl == grpb),(20,))
    A = disSMat[grpA_idx,:]
    B = A[:,grpB_idx]

    result = np.mean(B)

    if (grpa - grpb < 0.1):
        result = result * 20 / 19

    return result

def compareGroupsT(grpa,grpb,lbl,disSMat):
    #takes in two group indices, labels and dissimiliarity matrix, returns how similar they are as two groups
    grpA_idx = torch.where(lbl == grpa)[0]
    grpB_idx = torch.where(lbl == grpb)[0]
    A = disSMat[grpA_idx,:]
    B = A[:,grpB_idx]

    result = torch.mean(B)

    if (grpa - grpb < 0.1):
        result = result * 20 / 19

    return result

In [2]:
def obj_2_adj(fileName):
    #takes in file name, reads the .obj file and returns adjacency matrix scaled with edge values
    v, f = igl.read_triangle_mesh(fileName)
    distances = igl.all_pairs_distances(v,v,False)
    adJ = igl.adjacency_matrix(f).toarray() * distances
    return adJ

def obj_2_adj_noscale(fileName):
    #takes in file name, reads the .obj file and returns adjacency matrix
    v, f = igl.read_triangle_mesh(fileName)
    distances = igl.all_pairs_distances(v,v,False)
    adJ = igl.adjacency_matrix(f).toarray() * distances
    adJ = np.where(adJ != 0,1,0)
    return adJ

def adj_2_features(adj):
    #some arbitrary ad-hoc function for extracting features, for testing only

    #print([np.mean(adj),np.max(adj),np.std(adj)])
    #hist = np.histogram(adj,bins = np.arange(0,0.5,0.02),density=True)
    #generic_feat = np.array([np.mean(adj),np.max(adj),np.std(adj)])
    hist = np.histogram(adj,bins = 20,density=True)[0]
    #print(hist[0])
    return hist

# get same type of objects
def get_same_type(lbls):
    type = {}
    for i in range(600):
        lbl_idx = lbls[i].astype(int)
        if bool(type.get(lbl_idx)):
            type[lbl_idx].append(i)
        else:
            type[lbl_idx] = [i]

    return type

def get_test_train(type):
    rand_array = np.random.choice(16, size=(5,1), replace=False)
    test = {}
    for j in range(len(type)):
        for i in range(len(rand_array)):
            if bool(test.get(j)):
                test[j].append(type[j][rand_array.item(i)])
                del type[j][rand_array.item(i)]
            else:
                test[j] = [type[j][rand_array.item(i)]]
                del type[j][rand_array.item(i)]
    train = type
    return test,train


def prep_data(group):
    normed_adjMats_list = []
    node_sigs_list = []
    
    for i in range(30):
        for j in range(len(group[0])):
            fName = 'T' + str(train[i][j]) + '.obj'
            adj_noscale_read = obj_2_adj_noscale(mesh_dir + fName)
            adj_scaled_read = obj_2_adj(mesh_dir + fName)
    
            if (adj_scaled_read.shape[0] < 252):
                adj_noscale = np.empty([252,252])
                adj_scaled = np.empty([252,252])
            else:
                adj_noscale = adj_noscale_read
                adj_scaled = adj_scaled_read
    
            adj_normalized = adj_noscale / np.reshape(np.sum(adj_noscale,axis = 1),[adj_noscale.shape[0],1])
    
            #node level signal extraction
            node_degs = np.sum(adj_noscale,axis = 0)
            node_neigh_max = np.max(adj_scaled,axis = 0)
            node_neigh_min = np.min(adj_scaled,axis = 0)
            node_neigh_sum = np.sum(adj_scaled,axis = 0)
            node_neigh_mean = np.sum(adj_scaled,axis = 0)
    
            node_sig = np.stack([node_degs,node_neigh_max,node_neigh_min,node_neigh_sum,node_neigh_mean],axis = 1)
            node_sigs_list.append(node_sig)
    
            normed_adjMats_list.append(adj_normalized)
    return normed_adjMats_list, node_sigs_list

In [3]:
def form_label_matrix(label_mat, group):
    rows = []
    for i in range(30):
        for j in range(15):
            # delete test obj from label matrix
            rows.append(group[i][j])

    label_mat = label_mat[:,rows]
    label_mat = label_mat[rows, :]
    print(label_mat.shape)
    return label_mat